In [72]:
import sqlite3
import pandas as pd

# Specify the full path to the database file
db_path = '../data/database.sqlite'

# Connect to the database
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Now you can use the cursor to execute SQL commands
cursor.execute("SELECT sqlite_version();")
print("SQLite version:", cursor.fetchone())


SQLite version: ('3.45.3',)


In [73]:
# Query to list all tables
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

# Fetch all table names
tables = cursor.fetchall()

# Print the table names
print("Tables in the database:")
for table in tables:
    print(table[0])

Tables in the database:
Country
CountryNotes
Series
Indicators
SeriesNotes
Footnotes
G7_GDPs


In [74]:
# Define the query to select the first few rows
query = "SELECT * FROM Country LIMIT 5;"  # Replace 'your_table_name' with the name of your table

# Execute the query and load the results into a DataFrame
df = pd.read_sql_query(query, conn)

# Display the DataFrame
df

,CountryCode,ShortName,TableName,LongName,Alpha2Code,CurrencyUnit,SpecialNotes,Region,IncomeGroup,Wb2Code,...,GovernmentAccountingConcept,ImfDataDisseminationStandard,LatestPopulationCensus,LatestHouseholdSurvey,SourceOfMostRecentIncomeAndExpenditureData,VitalRegistrationComplete,LatestAgriculturalCensus,LatestIndustrialData,LatestTradeData,LatestWaterWithdrawalData
0,AFG,Afghanistan,Afghanistan,Islamic State of Afghanistan,AF,Afghan afghani,Fiscal year end: March 20; reporting period fo...,South Asia,Low income,AF,...,Consolidated central government,General Data Dissemination System (GDDS),1979,"Multiple Indicator Cluster Survey (MICS), 2010/11","Integrated household survey (IHS), 2008",,2013/14,,2013,2000
1,ALB,Albania,Albania,Republic of Albania,AL,Albanian lek,,Europe & Central Asia,Upper middle income,AL,...,Budgetary central government,General Data Dissemination System (GDDS),2011,"Demographic and Health Survey (DHS), 2008/09",Living Standards Measurement Study Survey (LSM...,Yes,2012,2011,2013,2006
2,DZA,Algeria,Algeria,People's Democratic Republic of Algeria,DZ,Algerian dinar,,Middle East & North Africa,Upper middle income,DZ,...,Budgetary central government,General Data Dissemination System (GDDS),2008,"Multiple Indicator Cluster Survey (MICS), 2012","Integrated household survey (IHS), 1995",,,2010,2013,2001
3,ASM,American Samoa,American Samoa,American Samoa,AS,U.S. dollar,,East Asia & Pacific,Upper middle income,AS,...,,,2010,,,Yes,2007,,,
4,ADO,Andorra,Andorra,Principality of Andorra,AD,Euro,,Europe & Central Asia,High income: nonOECD,AD,...,,,2011. Population data compiled from administra...,,,Yes,,,2006,


In [75]:
# Define the query to select the first few rows
query = "SELECT * FROM Indicators LIMIT 5;"  # Replace 'your_table_name' with the name of your table

# Execute the query and load the results into a DataFrame
df = pd.read_sql_query(query, conn)

# Display the DataFrame
df

,CountryName,CountryCode,IndicatorName,IndicatorCode,Year,Value
0,Arab World,ARB,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,1960,1.335609e+02
1,Arab World,ARB,Age dependency ratio (% of working-age populat...,SP.POP.DPND,1960,8.779760e+01
2,Arab World,ARB,"Age dependency ratio, old (% of working-age po...",SP.POP.DPND.OL,1960,6.634579e+00
3,Arab World,ARB,"Age dependency ratio, young (% of working-age ...",SP.POP.DPND.YG,1960,8.102333e+01
4,Arab World,ARB,Arms exports (SIPRI trend indicator values),MS.MIL.XPRT.KD,1960,3.000000e+06


In [76]:
# Define the query
query = """
SELECT DISTINCT IndicatorName
FROM Indicators
WHERE IndicatorName LIKE 'GDP%';
"""

# Execute the query and load the results into a DataFrame
df = pd.read_sql_query(query, conn)

# Display the DataFrame
print(df)

                                        IndicatorName
0                                  GDP (constant LCU)
1                                   GDP (current LCU)
2            GDP at market prices (constant 2005 US$)
3                  GDP at market prices (current US$)
4          GDP deflator (base year varies by country)
5                               GDP growth (annual %)
6                  GDP per capita (constant 2005 US$)
7                       GDP per capita (constant LCU)
8                        GDP per capita (current LCU)
9                        GDP per capita (current US$)
10                   GDP per capita growth (annual %)
11  GDP per capita, PPP (constant 2011 internation...
12      GDP per capita, PPP (current international $)
13      GDP per person employed (constant 1990 PPP $)
14  GDP per unit of energy use (PPP $ per kg of oi...
15  GDP per unit of energy use (constant 2011 PPP ...
16           GDP, PPP (constant 2011 international $)
17                 GDP, PPP 

In [77]:
# Define the query
query = """
SELECT *
FROM Indicators
WHERE IndicatorName IN ('GDP at market prices (current US$)', 'GDP per capita (current US$)');
"""

# Execute the query and load the results into a DataFrame
df = pd.read_sql_query(query, conn)

# Display the DataFrame
df

,CountryName,CountryCode,IndicatorName,IndicatorCode,Year,Value
0,Caribbean small states,CSS,GDP at market prices (current US$),NY.GDP.MKTP.CD,1960,1.917148e+09
1,East Asia & Pacific (all income levels),EAS,GDP at market prices (current US$),NY.GDP.MKTP.CD,1960,1.530501e+11
2,East Asia & Pacific (developing only),EAP,GDP at market prices (current US$),NY.GDP.MKTP.CD,1960,8.007443e+10
3,Euro area,EMU,GDP at market prices (current US$),NY.GDP.MKTP.CD,1960,2.453780e+11
4,Europe & Central Asia (all income levels),ECS,GDP at market prices (current US$),NY.GDP.MKTP.CD,1960,4.326820e+11
...,...,...,...,...,...,...
20684,Vanuatu,VUT,GDP per capita (current US$),NY.GDP.PCAP.CD,2014,3.147964e+03
20685,Vietnam,VNM,GDP per capita (current US$),NY.GDP.PCAP.CD,2014,2.052294e+03
20686,West Bank and Gaza,WBG,GDP per capita (current US$),NY.GDP.PCAP.CD,2014,2.965904e+03
20687,Zambia,ZMB,GDP per capita (current US$),NY.GDP.PCAP.CD,2014,1.721623e+03


In [78]:
# Define the updated query
query = """
SELECT *
FROM Indicators
WHERE IndicatorName IN ('GDP at market prices (current US$)', 'GDP per capita (current US$)')
AND CountryCode IN ('USA', 'CAN', 'FRA', 'DEU', 'ITA', 'JPN', 'GBR', 'CHN', 'IND');
"""

# Execute the query and load the results into a DataFrame
df = pd.read_sql_query(query, conn)

# Display the DataFrame
df

,CountryName,CountryCode,IndicatorName,IndicatorCode,Year,Value
0,Canada,CAN,GDP at market prices (current US$),NY.GDP.MKTP.CD,1960,4.109345e+10
1,China,CHN,GDP at market prices (current US$),NY.GDP.MKTP.CD,1960,5.918412e+10
2,France,FRA,GDP at market prices (current US$),NY.GDP.MKTP.CD,1960,6.265147e+10
3,India,IND,GDP at market prices (current US$),NY.GDP.MKTP.CD,1960,3.767927e+10
4,Italy,ITA,GDP at market prices (current US$),NY.GDP.MKTP.CD,1960,4.038529e+10
...,...,...,...,...,...,...
965,India,IND,GDP per capita (current US$),NY.GDP.PCAP.CD,2014,1.581511e+03
966,Italy,ITA,GDP per capita (current US$),NY.GDP.PCAP.CD,2014,3.490850e+04
967,Japan,JPN,GDP per capita (current US$),NY.GDP.PCAP.CD,2014,3.619442e+04
968,United Kingdom,GBR,GDP per capita (current US$),NY.GDP.PCAP.CD,2014,4.633198e+04


In [79]:
# Define the query to create the new table with the filtered data
query = """
CREATE TABLE IF NOT EXISTS G7_GDPs AS
SELECT *
FROM Indicators
WHERE IndicatorName IN ('GDP at market prices (current US$)', 'GDP per capita (current US$)')
AND CountryCode IN ('USA', 'CAN', 'FRA', 'DEU', 'ITA', 'JPN', 'GBR', 'CHN', 'IND');
"""

# Execute the query
cursor.execute(query)

In [80]:
# Define the query
query = """
SELECT * FROM G7_GDPs
"""

# Execute the query and load the results into a DataFrame
df = pd.read_sql_query(query, conn)

# Display the DataFrame
df

,CountryName,CountryCode,IndicatorName,IndicatorCode,Year,Value
0,Canada,CAN,GDP at market prices (current US$),NY.GDP.MKTP.CD,1960,4.109345e+10
1,China,CHN,GDP at market prices (current US$),NY.GDP.MKTP.CD,1960,5.918412e+10
2,France,FRA,GDP at market prices (current US$),NY.GDP.MKTP.CD,1960,6.265147e+10
3,India,IND,GDP at market prices (current US$),NY.GDP.MKTP.CD,1960,3.767927e+10
4,Italy,ITA,GDP at market prices (current US$),NY.GDP.MKTP.CD,1960,4.038529e+10
...,...,...,...,...,...,...
965,India,IND,GDP per capita (current US$),NY.GDP.PCAP.CD,2014,1.581511e+03
966,Italy,ITA,GDP per capita (current US$),NY.GDP.PCAP.CD,2014,3.490850e+04
967,Japan,JPN,GDP per capita (current US$),NY.GDP.PCAP.CD,2014,3.619442e+04
968,United Kingdom,GBR,GDP per capita (current US$),NY.GDP.PCAP.CD,2014,4.633198e+04


In [81]:
# Close the connection when done
conn.close()